In [1]:
# импортируем нужные библиотеки
from lxml import html
import requests
from pprint import pprint
import csv
import pandas as pd

In [2]:
url='https://tt.sport-liga.pro/'

# задаем user-agent
header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'
    }

# просьба ввести данные пользователя из консоли (В РАЗРАБОТКЕ)
date = input('Введите дату в формате DD:MM:YYYY ')
day, month, year = date.split('.')
param = {
    'year': year,
    'month': month,
    'day': day
}

# param = {
#     'year': '2024',
#     'month': '05',
#     'day': '15'
# }


url_tour = 'tours/'
# url_tour = 'tours/34256'
# url_tour = 'tours/34279'

session = requests.Session()

In [3]:
# делаем get-запрос
response = session.get(url+url_tour, params=param, headers=header)
# проверяем статус-код
if response.status_code < 400:
    # если да, печатаем статус код и представляем ответ в виде дерева html
    print(response.status_code)
    tree = html.fromstring(response.text)
else:
    print('Что-то пошло не так')

200


In [4]:
# находим нужную таблицу на странице
table = tree.xpath('//table[@class="table"]//tr')
# находим ссылки на переход к просмотру турнира
ref = tree.xpath('.//td[@class="tournament-name"]/a/@href')

In [5]:
tables = tree.xpath('//td[@class="tournament-name"]/a/text()')
tables = [el.split(' ')[1][:-1] for el in tables]
time = tree.xpath('//td[@class="tournament-date"]/text()')
time = [el.split(' ')[-1] for el in time]

In [6]:
tours = []
for row, item, i in zip(table[1:], ref, range(len(tables))):
    res = session.get(url+item, headers=header)
    tour = html.fromstring(res.text)

    player1 = tour.xpath('.//td[@class="right"]/a/text()')
    player2 = tour.xpath('.//td[@class="left"]/a/text()')
    score = tour.xpath('.//td[@class="score"]//a/text()')
    url_match = tour.xpath('.//td[@class="score"]//a/@href')
    if len(tour.xpath('//table')) > 3:
        score_set = tour.xpath('.//small[@class="nowrap"]/text()')
        for i in range(len(score)):
            if score[i] == "- : -":
                score_set.insert(i, 'не сыгран')
    else: score_set = ['не сыгран' for _ in range(len(score))]
        

    tours_info = {
        'table': tables[i],
        'time': time[i],
        'player1': player1,
        'player2': player2,
        'score': score ,
        'url_match': url_match,
        'score_set': score_set
    }

    tours.append(tours_info)

In [7]:
for el in tours:
    print(el)

{'table': 'А6', 'time': '23:45', 'player1': ['Обухов В', 'Илюхин Е', 'Мамека М', 'Обухов В', 'Кутузов О', 'Мамека М', 'Анисимов Е', 'Илюхин Е', 'Анисимов Е', 'Кутузов О'], 'player2': ['Мамека М', 'Анисимов Е', 'Кутузов О', 'Илюхин Е', 'Анисимов Е', 'Илюхин Е', 'Обухов В', 'Кутузов О', 'Мамека М', 'Обухов В'], 'score': ['- : -', '- : -', '- : -', '- : -', '- : -', '- : -', '- : -', '- : -', '- : -', '- : -'], 'url_match': ['games/324833', 'games/324834', 'games/324835', 'games/324836', 'games/324837', 'games/324838', 'games/324839', 'games/324840', 'games/324841', 'games/324842'], 'score_set': ['не сыгран', 'не сыгран', 'не сыгран', 'не сыгран', 'не сыгран', 'не сыгран', 'не сыгран', 'не сыгран', 'не сыгран', 'не сыгран']}
{'table': 'А4', 'time': '23:45', 'player1': ['Шершнев А', 'Лбов Ю', 'Слесарев В', 'Шершнев А', 'Федоров А', 'Слесарев В', 'Беспалов Н', 'Лбов Ю', 'Беспалов Н', 'Федоров А'], 'player2': ['Слесарев В', 'Беспалов Н', 'Федоров А', 'Лбов Ю', 'Беспалов Н', 'Лбов Ю', 'Шершне

In [8]:
df = pd.DataFrame(columns=['player1', 'player2', 'score', 'url_match', 'score_set'])
for el in tours:
    df_el = pd.DataFrame(el)
    df = df.append(df_el, ignore_index=True)
df['url_match'] = url + df['url_match']

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_1452\4205032851.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_el, ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_1452\4205032851.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_el, ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_1452\4205032851.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_el, ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_1452\4205032851.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_el, ignore_index=True)
C:\Users\Lenovo\AppData\Loca

In [9]:
df

,player1,player2,score,url_match,score_set,table,time
0,Обухов В,Мамека М,- : -,https://tt.sport-liga.pro/games/324833,не сыгран,А6,23:45
1,Илюхин Е,Анисимов Е,- : -,https://tt.sport-liga.pro/games/324834,не сыгран,А6,23:45
2,Мамека М,Кутузов О,- : -,https://tt.sport-liga.pro/games/324835,не сыгран,А6,23:45
3,Обухов В,Илюхин Е,- : -,https://tt.sport-liga.pro/games/324836,не сыгран,А6,23:45
4,Кутузов О,Анисимов Е,- : -,https://tt.sport-liga.pro/games/324837,не сыгран,А6,23:45
...,...,...,...,...,...,...,...
151,Василевский К,Рохлин И,- : -,https://tt.sport-liga.pro/games/324776,не сыгран,А5,07:30
152,Сергеевич И,Василевский К,- : -,https://tt.sport-liga.pro/games/324777,не сыгран,А5,07:30
153,Рохлин И,Дворниченко Ю,- : -,https://tt.sport-liga.pro/games/324778,не сыгран,А5,07:30
154,Дворниченко Ю,Василевский К,- : -,https://tt.sport-liga.pro/games/324779,не сыгран,А5,07:30


In [10]:
df.to_csv(f'tours_{date}.csv', index=False)

In [11]:
ta = input('Выберите стол: ')

df = df[df['table'] == ta]
df = df[df['time'] > '15']
df = df[df['time'] <= '20']
df

,player1,player2,score,url_match,score_set,table,time


In [12]:
df.to_csv(f'tours_{date}_change.csv', index=False)

In [13]:
df['url_match']

Series([], Name: url_match, dtype: object)